In [1]:
!pip install torchdata==0.7.1 --quiet
!pip install dgl -f https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 4.8 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtune 0.6.1 requires torchdata==0.11.0, but you have torchdata 0.7.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.2/483.2 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━

In [2]:
!pip install tensorboardX pandas numpy networkx tqdm scikit-learn scipy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.6 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import networkx as nx
import dgl
import math
import os
import sys
from tqdm import tqdm
from scipy.sparse.linalg import eigs
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import warnings

warnings.filterwarnings('ignore')

# --- Thiết lập Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Utils cho GCN (Bổ sung phần thiếu) ---
def scaled_Laplacian(W):
    '''
    Tính toán Scaled Laplacian matrix cho ChebNet
    '''
    assert W.shape[0] == W.shape[1]
    D = torch.diag(torch.sum(W, axis=1))
    L = D - W
    lambda_max = 2.0  # Giả định, hoặc tính bằng eigs(L, k=1, which='LM')[0].real
    if isinstance(W, torch.Tensor):
        return (2 * L) / lambda_max - torch.eye(W.shape[0]).to(W.device)
    else:
        return (2 * L) / lambda_max - np.identity(W.shape[0])

def cheb_polynomial(L_tilde, K):
    '''
    Tính đa thức Chebyshev
    '''
    N = L_tilde.shape[0]
    cheb_polynomials = [torch.eye(N).to(L_tilde.device), L_tilde.clone()]
    for i in range(2, K):
        cheb_polynomials.append(2 * L_tilde * cheb_polynomials[i - 1] - cheb_polynomials[i - 2])
    return cheb_polynomials

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

# --- Paste toàn bộ các class từ file bạn cung cấp vào đây ---
# Tôi sẽ rút gọn lại để tập trung vào Encoder_GRU_SubMAGCN vì chúng ta dùng nó làm encoder chính.

class GCN(nn.Module):
    def __init__(self, dim_in, dim_out, order_K, device, in_drop=0.0, gcn_drop=0.0, residual=False):
        super(GCN, self).__init__()
        self.DEVICE = device
        self.order_K = order_K
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.Theta = nn.ParameterList([nn.Parameter(torch.FloatTensor(dim_in, dim_out)) for _ in range(order_K)])
        self.weights = nn.Parameter(torch.FloatTensor(size=(dim_out, dim_out)))
        self.biases = nn.Parameter(torch.FloatTensor(size=(dim_out,)))
        self._in_drop = in_drop
        self._gcn_drop = gcn_drop
        self._residual = residual
        self.linear = nn.Linear(dim_in, dim_out)
        self.reset_parameters()

    def reset_parameters(self):
        for param in self.parameters():
            if len(param.shape) > 1:
                nn.init.xavier_uniform_(param)
            else:
                nn.init.zeros_(param)

    def forward(self, x, L_tilde):
        # x: (batch, nodes, features)
        batch_size, num_of_vertices, in_channels = x.shape
        cheb_polynomials = cheb_polynomial(L_tilde, self.order_K)
        
        output = torch.zeros(batch_size, num_of_vertices, self.dim_out).to(self.DEVICE)
        
        x0 = x
        if self._in_drop != 0:
            x = torch.dropout(x, 1.0 - self._in_drop, train=self.training)
            
        for k in range(self.order_K):
            # Cheb poly: (nodes, nodes)
            # x: (batch, nodes, feat) -> permute(0,2,1) -> (batch, feat, nodes)
            # Support = x * T_k
            support = torch.matmul(x.permute(0, 2, 1), cheb_polynomials[k]).permute(0, 2, 1)
            output = output + torch.matmul(support, self.Theta[k])
            
        output = torch.matmul(output, self.weights) + self.biases
        res = F.relu(output)
        
        if self._gcn_drop != 0.0:
            res = torch.dropout(res, 1.0 - self._gcn_drop, train=self.training)
        if self._residual:
            x0 = self.linear(x0)
            res = res + x0
        return res

class MRA_GCN_Simple(nn.Module):
    # Simplified version of MRA for the adapter
    def __init__(self, dim_in, dim_out, range_K, device):
        super(MRA_GCN_Simple, self).__init__()
        self.DEVICE = device
        self.GCN = GCN(dim_in, dim_out, range_K, device)
        self.W_a = nn.Parameter(torch.FloatTensor(dim_out, dim_out))
        self.U = nn.Parameter(torch.FloatTensor(dim_out))
        nn.init.xavier_uniform_(self.W_a)
        nn.init.uniform_(self.U)

    def forward(self, X, L_tilde):
        # X: (batch, nodes, feat)
        gcn_out = self.GCN(X, L_tilde) # (batch, nodes, dim_out)
        
        # Attention Mechanism (Simplified for node-level)
        # e = gcn_out * Wa * U
        score = torch.matmul(torch.matmul(gcn_out, self.W_a), self.U) # (batch, nodes)
        alpha = F.softmax(score, dim=1).unsqueeze(-1) # (batch, nodes, 1)
        
        # Weighted context
        # context = torch.sum(gcn_out * alpha, dim=1, keepdim=True) # (batch, 1, dim_out)
        # return gcn_out + context # Residual connection style
        return gcn_out

class Encoder_GRU_Custom(nn.Module):
    def __init__(self, dim_in, dim_out, range_K, device):
        super(Encoder_GRU_Custom, self).__init__()
        self.DEVICE = device
        self.dim_in = dim_in
        self.dim_out = dim_out
        
        # Gates using GCN
        self.gate_conv = MRA_GCN_Simple(dim_in + dim_out, dim_out * 2, range_K, device)
        self.update_conv = MRA_GCN_Simple(dim_in + dim_out, dim_out, range_K, device)

    def forward(self, x, hx, L_tilde):
        # x: (batch, nodes, dim_in)
        # hx: (batch, nodes, dim_out)
        combined = torch.cat((x, hx), dim=-1)
        
        gates = self.gate_conv(combined, L_tilde)
        reset_gate, update_gate = torch.split(gates, self.dim_out, dim=-1)
        reset_gate = torch.sigmoid(reset_gate)
        update_gate = torch.sigmoid(update_gate)
        
        combined_r = torch.cat((x, reset_gate * hx), dim=-1)
        candidate = torch.tanh(self.update_conv(combined_r, L_tilde))
        
        hy = update_gate * hx + (1 - update_gate) * candidate
        return hy

In [5]:
class MADGCN(nn.Module):
    def __init__(self, 
                 poi_feat_dim, 
                 temp_feat_dim, 
                 ext_feat_dim, 
                 hidden_dim=32, 
                 K_hop=3):
        super(MADGCN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.device = device
        
        # Feature embeddings
        self.spatial_linear = nn.Linear(poi_feat_dim, hidden_dim)
        self.temporal_linear = nn.Linear(temp_feat_dim, hidden_dim) 
        
        # Encoder (Time series processing)
        self.encoder = Encoder_GRU_Custom(dim_in=hidden_dim, dim_out=hidden_dim, range_K=K_hop, device=device)
        
        # External features processing
        self.ext_linear = nn.Linear(ext_feat_dim, 16)
        
        # Final prediction
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim + 16, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid() 
        )

    def forward(self, g, spatial_features, temporal_features, external_features):
        """
        Sửa lỗi FileNotFoundError bằng cách tính Adjacency Matrix thủ công
        """
        # --- BẮT ĐẦU ĐOẠN SỬA LỖI ---
        # Thay vì gọi g.adjacency_matrix() gây lỗi thư viện C++
        # Chúng ta lấy danh sách cạnh (src, dst) và tự tạo ma trận
        src, dst = g.edges()
        N = g.num_nodes()
        
        # Tạo ma trận kề dense (N x N)
        adj = torch.zeros((N, N), device=self.device)
        adj[src, dst] = 1 # Gán trọng số cạnh là 1
        
        # Tính Laplacian từ ma trận tự tạo này
        L_tilde = scaled_Laplacian(adj) 
        # --- KẾT THÚC ĐOẠN SỬA LỖI ---
        
        # 2. Xử lý đầu vào (Input Processing)
        # Chuyển temporal features về dạng (Time, Nodes, Feat)
        ts_data = temporal_features.permute(2, 0, 1) 
        
        # Khởi tạo hidden state
        hx = torch.zeros(1, N, self.hidden_dim).to(self.device) 
        
        # 3. Encoder Loop (GRU)
        for t in range(ts_data.shape[0]):
            x_t = ts_data[t].unsqueeze(0) # (1, Nodes, 1)
            x_emb = self.temporal_linear(x_t) # (1, Nodes, Hidden)
            hx = self.encoder(x_emb, hx, L_tilde)
        
        # hx shape: (1, Nodes, Hidden) -> squeeze -> (Nodes, Hidden)
        node_features = hx.squeeze(0) 
        
        # Lấy feature của các node trung tâm (Target Nodes)
        # Trong DGL batch, các graph được nối tiếp nhau. Cần lấy node đầu tiên của mỗi subgraph.
        batch_num_nodes_list = g.batch_num_nodes().tolist()
        target_indices = []
        current_idx = 0
        for n_nodes in batch_num_nodes_list:
            target_indices.append(current_idx)
            current_idx += n_nodes
            
        target_node_feats = node_features[target_indices] # (Batch_Size, Hidden)
        
        # 4. Kết hợp External Features
        ext_emb = self.ext_linear(external_features) # (Batch_Size, 16)
        
        final_emb = torch.cat((target_node_feats, ext_emb), dim=1)
        
        output = self.output_layer(final_emb)
        return output

In [6]:
# --- Cấu hình ---
# Giả lập config.json
config = {
    "K_hop": 2, # DSTGCN dùng subgraph, K nhỏ
    "batch_size": 32,
    "spatial_features_mean": [0]*22, # Placeholder, cần load từ file thật
    "spatial_features_std": [1]*22,
    "temporal_features_mean": [0],
    "temporal_features_std": [1],
    "external_features_mean": [0]*43,
    "external_features_std": [1]*43,
    "poi_features_number": 22,
    "temporal_features_number": 1,
    "external_features_number": 43, # 38 weather + 5 time
    "epochs": 50,
    "learning_rate": 0.001
}

# --- Helper Functions cho Data Loading ---
longitudeMin, longitudeMax = 116.09608, 116.71040
latitudeMin, latitudeMax = 39.69086, 40.17647
widthSingle = 0.01 / math.cos(latitudeMin / 180 * math.pi) / 5
heightSingle = 0.01 / 5

def collate_fn(batch):
    ret = list()
    for idx, item in enumerate(zip(*batch)):
        if isinstance(item[0], torch.Tensor):
            if idx < 3: 
                ret.append(torch.cat(item))
            else: 
                ret.append(torch.stack(item))
        elif isinstance(item[0], dgl.DGLGraph):
            ret.append(dgl.batch(item))
    return tuple(ret)

def fill_speed(speed_data):
    # (Giữ nguyên logic fill missing data của DSTGCN)
    speed_data = speed_data.resample(rule="1H").mean()
    speed_data = speed_data.fillna(method='ffill').fillna(method='bfill')
    return speed_data

class AccidentDataset(Dataset):
    def __init__(self, k_order, network, node_attr, accident, weather, speed, sf_scaler=None, tf_scaler=None, ef_scaler=None):
        self.k_order = k_order
        self.network = network
        self.nodes = node_attr
        self.accident = accident
        self.weather = weather
        self.speed = speed
        self.sf_scaler = sf_scaler
        self.tf_scaler = tf_scaler
        self.ef_scaler = ef_scaler

    def __getitem__(self, sample_id):
        # Trích xuất dữ liệu cho 1 mẫu tai nạn
        _, _, accident_time, node_id, target = self.accident.iloc[sample_id]
        
        # Tạo Subgraph
        neighbors = nx.single_source_shortest_path_length(self.network, node_id, cutoff=self.k_order)
        neighbors_list = [node_id] + sorted([n for n in neighbors.keys() if n != node_id])
        
        sub_graph_nx = nx.subgraph(self.network, neighbors_list)
        relabel_map = {old: new for new, old in enumerate(neighbors_list)}
        sub_graph_nx = nx.relabel_nodes(sub_graph_nx, relabel_map)
        sub_graph_nx.add_edges_from([(v, v) for v in sub_graph_nx.nodes]) # Self-loop
        g = dgl.from_networkx(sub_graph_nx)

        # Lấy Features
        date_range = pd.date_range(end=accident_time.strftime("%Y%m%d %H"), freq="1H", periods=24)
        
        # Spatial
        selected_nodes = self.nodes.loc[neighbors_list]
        spatial_features = np.array(selected_nodes['spatial_features'].tolist())

        # Temporal (Speed) - Mapping Grid ID
        x_ids = np.floor((selected_nodes['XCoord'].values - longitudeMin) / widthSingle).astype(np.int64)
        y_ids = np.floor((selected_nodes['YCoord'].values - latitudeMin) / heightSingle).astype(np.int64)
        grid_keys = [f'{y},{x}' for y, x in zip(y_ids, x_ids)]
        
        # Handle missing grids safely
        temp_list = []
        for key in grid_keys:
            if key in self.speed.columns:
                temp_list.append(self.speed.loc[date_range, key].values)
            else:
                temp_list.append(np.zeros(24)) # Default 0 if missing
        temporal_features = np.array(temp_list)

        # External
        weather_data = self.weather.loc[date_range[-1]].tolist() if date_range[-1] in self.weather.index else [0]*38
        external_features = weather_data + [accident_time.month, accident_time.day, accident_time.dayofweek, accident_time.hour, int(accident_time.dayofweek >= 5)]

        # Scaling
        if self.sf_scaler: spatial_features = (spatial_features - self.sf_scaler[0]) / self.sf_scaler[1]
        if self.tf_scaler: temporal_features = (temporal_features - self.tf_scaler[0]) / self.tf_scaler[1]
        if self.ef_scaler: external_features = (external_features - self.ef_scaler[0]) / self.ef_scaler[1]

        # To Tensor
        # Spatial: (Nodes, Feat)
        # Temporal: (Nodes, 1, Time)
        spatial_features = torch.tensor(spatial_features).float()
        temporal_features = torch.tensor(temporal_features).unsqueeze(1).float()
        external_features = torch.tensor(external_features).float()
        target = torch.tensor(target).float()

        return g, spatial_features, temporal_features, external_features, target

    def __len__(self):
        return len(self.accident)

def get_data_loaders(config):
    # Giả định đường dẫn file, bạn cần thay đổi cho đúng môi trường Colab/Kaggle
    path_prefix = "/kaggle/input/dstgcn-dataset/" # Hoặc "./data/"
    try:
        import pickle
        with open(path_prefix + 'beijing_roadnet.gpickle', 'rb') as f:
            network = pickle.load(f)
        nodes = pd.read_hdf(path_prefix + 'edges_data.h5')
        weather = pd.read_hdf(path_prefix + 'weather.h5')
        speed = fill_speed(pd.read_hdf(path_prefix + 'all_grids_speed.h5'))
        accident_path = path_prefix + 'accident.h5'
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

    dls = {}
    for key in ['train', 'validate', 'test']:
        accident = pd.read_hdf(accident_path, key=key)
        # Sử dụng 1 phần dữ liệu để test code nếu cần (vd: accident.iloc[:100])
        ds = AccidentDataset(
            config['K_hop'], network, nodes, accident, weather, speed,
            sf_scaler=(np.array(config['spatial_features_mean']), np.array(config['spatial_features_std'])),
            tf_scaler=(np.array(config['temporal_features_mean']), np.array(config['temporal_features_std'])),
            ef_scaler=(np.array(config['external_features_mean']), np.array(config['external_features_std']))
        )
        dls[key] = DataLoader(ds, batch_size=config['batch_size'], shuffle=(key=='train'), collate_fn=collate_fn, num_workers=0)
    return dls

In [7]:
def train_madgcn_model(model, data_loaders, optimizer, criterion, epochs):
    model.to(device)
    best_rmse = float('inf')
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        for phase in ['train', 'validate', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            predictions = []
            targets = []
            
            # Sử dụng tqdm nếu chạy interactive
            # loader = tqdm(data_loaders[phase], desc=phase)
            loader = data_loaders[phase]
            
            for g, s_feat, t_feat, e_feat, y_true in loader:
                g = g.to(device)
                s_feat = s_feat.to(device)
                t_feat = t_feat.to(device)
                e_feat = e_feat.to(device)
                y_true = y_true.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(g, s_feat, t_feat, e_feat)
                    output = output.squeeze()
                    
                    loss = criterion(output, y_true)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * y_true.size(0)
                
                predictions.extend(output.detach().cpu().numpy())
                targets.extend(y_true.detach().cpu().numpy())
            
            epoch_loss = running_loss / len(data_loaders[phase].dataset)
            predictions = np.array(predictions)
            targets = np.array(targets)
            
            # --- Tính Metrics ---
            mae = mean_absolute_error(targets, predictions)
            rmse = np.sqrt(mean_squared_error(targets, predictions))
            pcc, _ = pearsonr(predictions.flatten(), targets.flatten())
            
            print(f"{phase.upper()} | Loss: {epoch_loss:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f} | PCC: {pcc:.4f}")
            
            if phase == 'test' and rmse < best_rmse:
                best_rmse = rmse
                # torch.save(model.state_dict(), 'best_madgcn.pth')

In [8]:
# 1. Load Data
print("Loading data...")
loaders = get_data_loaders(config)

if loaders:
    # 2. Khởi tạo Model
    print("Initializing MADGCN Model...")
    model = MADGCN(
        poi_feat_dim=config['poi_features_number'],
        temp_feat_dim=config['temporal_features_number'], # Usually 1
        ext_feat_dim=config['external_features_number'],
        hidden_dim=32,
        K_hop=config['K_hop']
    )
    
    # 3. Setup Optimizer & Loss
    # Bài toán dự đoán rủi ro (0-1), có thể dùng MSE hoặc BCELoss
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.MSELoss() 
    
    # 4. Train
    print("Start Training...")
    train_madgcn_model(model, loaders, optimizer, criterion, epochs=config['epochs'])
else:
    print("Cannot create dataloaders. Check data paths.")

Loading data...
Initializing MADGCN Model...
Start Training...
Epoch 1/50
TRAIN | Loss: 0.2082 | MAE: 0.4231 | RMSE: 0.4563 | PCC: 0.4107
VALIDATE | Loss: 0.1984 | MAE: 0.4067 | RMSE: 0.4454 | PCC: 0.4820
TEST | Loss: 0.1974 | MAE: 0.4076 | RMSE: 0.4443 | PCC: 0.4922
Epoch 2/50
TRAIN | Loss: 0.1800 | MAE: 0.3707 | RMSE: 0.4243 | PCC: 0.5308
VALIDATE | Loss: 0.1951 | MAE: 0.3775 | RMSE: 0.4418 | PCC: 0.5185
TEST | Loss: 0.1918 | MAE: 0.3745 | RMSE: 0.4379 | PCC: 0.5351
Epoch 3/50
TRAIN | Loss: 0.1662 | MAE: 0.3439 | RMSE: 0.4077 | PCC: 0.5807
VALIDATE | Loss: 0.1823 | MAE: 0.3490 | RMSE: 0.4270 | PCC: 0.5437
TEST | Loss: 0.1797 | MAE: 0.3460 | RMSE: 0.4239 | PCC: 0.5538
Epoch 4/50
TRAIN | Loss: 0.1588 | MAE: 0.3249 | RMSE: 0.3985 | PCC: 0.6045
VALIDATE | Loss: 0.1543 | MAE: 0.3219 | RMSE: 0.3929 | PCC: 0.6241
TEST | Loss: 0.1582 | MAE: 0.3267 | RMSE: 0.3977 | PCC: 0.6120
Epoch 5/50
TRAIN | Loss: 0.1514 | MAE: 0.3069 | RMSE: 0.3892 | PCC: 0.6282
VALIDATE | Loss: 0.1627 | MAE: 0.3272 | RM

In [9]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
import torch

def evaluate_final_metrics(model, test_loader, device):
    print("--- Bắt đầu đánh giá trên toàn bộ tập Test ---")
    
    model.eval()  # Chuyển model sang chế độ đánh giá (không dropout/grad)
    predictions = []
    targets = []
    
    # Tắt tính toán gradient để tiết kiệm bộ nhớ và tăng tốc
    with torch.no_grad():
        for g, s_feat, t_feat, e_feat, y_true in test_loader:
            # Chuyển dữ liệu sang thiết bị (GPU/CPU)
            g = g.to(device)
            s_feat = s_feat.to(device)
            t_feat = t_feat.to(device)
            e_feat = e_feat.to(device)
            y_true = y_true.to(device)
            
            # Dự đoán
            output = model(g, s_feat, t_feat, e_feat)
            output = output.squeeze()  # Chuyển shape (B, 1) -> (B,)
            
            # Lưu lại kết quả (chuyển về CPU và numpy)
            predictions.extend(output.cpu().numpy())
            targets.extend(y_true.cpu().numpy())
            
    # Chuyển list thành numpy array
    predictions = np.array(predictions)
    targets = np.array(targets)
    
    # Tính toán các metrics
    mae = mean_absolute_error(targets, predictions)
    rmse = np.sqrt(mean_squared_error(targets, predictions))
    pcc, p_value = pearsonr(predictions.flatten(), targets.flatten())
    
    # Hiển thị kết quả đẹp
    print("\n" + "="*40)
    print(f"{'KẾT QUẢ ĐÁNH GIÁ TỔNG HỢP (TEST SET)':^40}")
    print("="*40)
    print(f"{'Metric':<10} | {'Giá trị':<20}")
    print("-" * 40)
    print(f"{'MAE':<10} | {mae:.6f}")
    print(f"{'RMSE':<10} | {rmse:.6f}")
    print(f"{'PCC':<10} | {pcc:.6f}")
    print("="*40)
    
    return {'mae': mae, 'rmse': rmse, 'pcc': pcc}

# --- THỰC THI ---
# Kiểm tra xem loaders và model đã tồn tại chưa
if 'loaders' in globals() and 'model' in globals():
    final_results = evaluate_final_metrics(model, loaders['test'], device)
else:
    print("Lỗi: Bạn cần chạy các cell phía trên để khởi tạo 'loaders' và 'model' trước.")

--- Bắt đầu đánh giá trên toàn bộ tập Test ---

  KẾT QUẢ ĐÁNH GIÁ TỔNG HỢP (TEST SET)  
Metric     | Giá trị             
----------------------------------------
MAE        | 0.212302
RMSE       | 0.342545
PCC        | 0.734217
